In [2]:
import gzip
import json
import pandas as pd
import numpy as np
import re
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from multiprocessing import Pool
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq
from memory_profiler import memory_usage

### Before starting our ETL, we did a prior processing of the original databases with the tool "codebeautyfy.org/json-fixer"
### To obtain valid json files.

In [2]:
# Constante para la ruta de nuestros archivos [0] = Games [1] = reviews [2] = items

FILE_NAMES = ['.\\Datasets\\steam_games.json.gz','.\\Datasets\\user_reviews_fixed.json.gz','.\\Datasets\\users_items_fixed.json.gz']

In [3]:
def file_to_dataframe(file):

    """
    Create a pandas dataframe from a JSON file.

    Parameters
    ----------
    file : str
        The path or URL of the JSON file.

    Returns
    -------
    file_data_frame : pd.DataFrame
        The dataframe created from the JSON file.

    Raises
    ------
    ValueError
        If the JSON file is not valid or cannot be read.
    """
    try:
        file_data_frame = pd.read_json(file, compression='gzip',lines=True)
        return file_data_frame
    except ValueError:
        file_data_frame = pd.read_json(file, compression='gzip')
        return file_data_frame
    except:
        raise ValueError('Something went wrong, Dataframe not created')

In [4]:
# Cargamos los Json a cada una de las dataframes

steam_games_df = file_to_dataframe(FILE_NAMES[0])
user_reviews_df = file_to_dataframe(FILE_NAMES[1])
user_items_df = file_to_dataframe(FILE_NAMES[2])


Null Treatment for dataframes

In [5]:
steam_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [6]:
user_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [7]:
user_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [8]:
def drop_empty_rows(dataframe):

    """
    Drop rows from a dataframe that contain only missing values.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The dataframe to drop rows from.

    Returns
    -------
    pd.DataFrame
        The dataframe with empty rows dropped. The original dataframe is modified in place.
    """
    dataframe.dropna(how='all', inplace=True)
    return dataframe

steam_games_df = drop_empty_rows(steam_games_df)
user_reviews_df = drop_empty_rows(user_reviews_df)
user_items_df = drop_empty_rows(user_items_df)

### Removemos las columnas que no utilizaremos a futuro

In [9]:
# Se detecta que app_name y title son básicamente lo mismo, se decide dropear title porque tiene vacios y app_name no

def compare_name_title(steam_games_df):
    """
    Compare the app_name and title columns of a dataframe of Steam games.

    This function prints the number of null values in each column and displays
    a subset of the dataframe where the app_name and title are different and
    not null.

    Parameters
    ----------
    steam_games_df : pd.DataFrame
        The dataframe of Steam games to compare.

    Returns
    -------
    None
        The function does not return anything. It only prints and displays data.
    """ 
    
    print(f"Nulos en title : {steam_games_df['title'].isnull().sum()}")
    print(f"Nullos en app_name: {steam_games_df['app_name'].isnull().sum()}")
    
    compare_appname_title = steam_games_df[(steam_games_df['app_name'] != steam_games_df['title']) & (~steam_games_df['app_name'].isnull()) & (~steam_games_df['title'].isnull())]
    display(compare_appname_title[['app_name','title']])
    
compare_name_title(steam_games_df)

Nulos en title : 2050


Nullos en app_name: 2


,app_name,title
88390,Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock
88393,Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy
88419,Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3
88492,Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate
88494,Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East
...,...,...
120181,Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0
120182,Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!
120183,Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon
120208,Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm


In [10]:
# Dropeamos columnas que no usaremos a futuro
# steam_Games | reviews_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url 
#             | title :  verificamos que el title es lo mismo que el app_name pero con muchos vacíos así que decidimos dropearlos

# user_reviews | user_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url
# user_items | user_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url

# Limitación: Para fines del proyecto no se contempla pero podríamos mejorar la base de datos haciendo un webScrapping con las url

steam_games_df = steam_games_df.drop('reviews_url', axis=1)
steam_games_df = steam_games_df.drop('title', axis=1)

# Dropeamos los 2 nulos de app_name
steam_games_df = steam_games_df.dropna(subset=['app_name'])

user_items_df = user_items_df.drop('user_url',axis=1)
user_reviews_df = user_reviews_df.drop('user_url', axis=1)

## Chequeo de columnas numericas

In [11]:
# Steam | Price

not_numeric = []
def not_numeric_values(value):
    try : 
        num = float(value)
        return None
    except ValueError:
        not_numeric.append(value)
    except TypeError:
        not_numeric.append(-1)


def convert_price(price):
    if price in ['Free Demo','Free to Play','Free To Play','Play for Free!','Free', 'Install Now', 'Play WARMACHINE: Tactics Demo', 
                 'Free Mod','Install Theme','Third-party','Play Now','Free HITMAN™ Holiday Pack','Install Now','Play the Demo','Free to Try',
                 'Free Movie','Free to Use']:
        return 0
    elif price == 'Starting at $499.00':
        return 499.00
    elif price == 'Starting at $449.00':
        return 449.00
    else:
        return price

# We apply the price conversion to all the data of type String
steam_games_df.loc[:,'price'] = steam_games_df['price'].apply(convert_price)

# Function used to find the non-numeric data
steam_games_df['price'].apply(not_numeric_values)        
print(not_numeric) 

# Fill the empty data with the mean of the column
# Decided to use the mean because it is less affected by outliers
# Decided not to drop these data to avoid affecting future functions based on the price of games

steam_games_df['price'] = steam_games_df['price'].fillna(steam_games_df['price'].median())
steam_games_df.info()
steam_games_df

[]
<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28851 non-null  object 
 2   app_name      32133 non-null  object 
 3   url           32133 non-null  object 
 4   release_date  30067 non-null  object 
 5   tags          31971 non-null  object 
 6   specs         31464 non-null  object 
 7   price         32133 non-null  float64
 8   early_access  32133 non-null  float64
 9   id            32132 non-null  float64
 10  developer     28836 non-null  object 
dtypes: float64(3), object(8)
memory usage: 2.9+ MB


,publisher,genres,app_name,url,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [12]:
steam_games_df['release_date'] = pd.to_datetime(steam_games_df['release_date'], format='%Y-%m-%d', errors='coerce')
steam_games_df

,publisher,genres,app_name,url,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


## FEATURING ENGINEERING

#### Reviews Sentiment Analysis

In [13]:
# nltk.download("vader_lexicon")



In [14]:
def get_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(review)
    return sentiment['compound']

def get_user_sentiment(reviews):
    sentiments = []
    for review in reviews:
        sentiment = get_sentiment(review['review'])
        # sentiments.append(sentiment)
        dict_review = [{'Posted: ':review['posted'], 'Last_Edited':review['last_edited'],'item_id':review['item_id'], 'recommend':review['recommend'] , 'sentiment':sentiment}]
        sentiments.append(dict_review)
    # avg_sentiment = sum(sentiments)  # maybe solo la suma
    return sentiments

def add_sentiment_column(df):
    df['sentiment'] = user_reviews_df['reviews'].apply(get_user_sentiment)

def user_review_explode(review_df):
    review_df = review_df[['user_id','sentiment']]
    review_df = review_df.explode('sentiment')
    review_df = review_df.explode('sentiment')
    hold_user = review_df[['user_id']]
    hold_user = hold_user.reset_index(drop=True)
    sentiment_exploded = pd.json_normalize(review_df['sentiment'])
    output_df = pd.concat([hold_user,sentiment_exploded], axis=1, join='inner')
    return output_df

In [15]:
# user_reviews_df
add_sentiment_column(user_reviews_df)
user_reviews_df = user_review_explode(user_reviews_df)
user_reviews_df


,user_id,Posted:,Last_Edited,item_id,recommend,sentiment
0,76561197970982479,"Posted November 5, 2011.",,1250,True,0.8481
1,76561197970982479,"Posted July 15, 2011.",,22200,True,0.2263
2,76561197970982479,"Posted April 21, 2011.",,43110,True,0.9117
3,js41637,"Posted June 24, 2014.",,251610,True,0.9566
4,js41637,"Posted September 8, 2013.",,227300,True,0.9708
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,,70,True,0.5574
59329,76561198312638244,Posted July 8.,,362890,True,0.9786
59330,LydiaMorley,Posted July 3.,,273110,True,0.7827
59331,LydiaMorley,Posted July 20.,,730,True,0.5106


### User_Items

In [16]:
user_items_df

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982480,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...
88305,76561198323066619,22,76561198323066624,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,[]
88308,76561198329548331,7,76561198329548336,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [33]:
user_items_df.drop('steam_id', axis=1)
user_items_df.drop('items_count', axis=1)
user_items_df.to_parquet('borrar_user_items_df.parquet')

In [17]:
def find_cost(item_id):
    price_id = steam_games_df[['price','id']]
    print(price_id)
    algo = price_id[price_id['id'].isin(item_id)]
    print (algo)
    
def items_price(items_df):
    cost=0
    for user_item_list in items_df:
        for item in user_item_list:
            print(item['item_id'])
            cost = find_cost(item['item_id'])
        break
    # print(items_df)


In [18]:
items_price(user_items_df['items'])

10
        price        id
88310    4.99  761140.0
88311    0.00  643980.0
88312    0.00  670290.0
88313    0.99  767400.0
88314    2.99  773570.0
...       ...       ...
120440   1.99  773640.0
120441   4.99  733530.0
120442   1.99  610660.0
120443   4.99  658870.0
120444   4.99  681550.0

[32133 rows x 2 columns]


TypeError: only list-like objects are allowed to be passed to isin(), you passed a `str`

In [ ]:
steam_games_df[steam_games_df.isnull().all(axis=1)]

In [ ]:
# Sentiment_analysis
# nltk.download() ### 




In [ ]:
# user_reviews_df
# user_reviews = user_reviews_df['reviews']

# user_reviews_df


In [ ]:
# user_reviews_df = user_reviews_df.drop('reviews', axis=1)
# user_reviews_df

In [ ]:
steam_games_df

In [ ]:
steam_games_df

In [ ]:
steam_games_df

In [ ]:
# steam_games_df = steam_games_df.explode(['genres'])
# steam_games_df = steam_games_df.explode(['tags'])
# steam_games_df = steam_games_df.explode(['specs'])

In [ ]:
# dummies_genres = pd.get_dummies (steam_games_df['genres'], prefix='genres')
# dummies_tags = pd.get_dummies (steam_games_df['tags'], prefix='tags')
# dummies_specs = pd.get_dummies (steam_games_df['specs'], prefix='specs')


In [ ]:
## Se decide en las fechas faltantes usar un mes y día standard 01/ 01 ya que solo necesitamos precisamente el valor del año

# def fix_release_date(date):

In [ ]:
#Checking Columns

# publisher_data = steam_games_df['publisher'].unique()
# # genres_data = steam_games_df['genres'].unique()
# app_name_data = steam_games_df['app_name'].unique()
# release_date_data = steam_games_df['release_date'].unique()

# for i in release_date_data:
#     print(i)

In [ ]:
user_items_df

## PARQUETS

#### Query 1

In [ ]:
query1_parquet = steam_games_df[['developer','price','release_date']]
query1_parquet.to_parquet('query1.parquet')

#### Query 2

In [ ]:
query_2_steam_parquet = steam_games_df[['price','id']]
query_2_steam_parquet.to_parquet('query_2_steam.parquet')

query_2_items_parquet = user_items_df[['user_id','items']]
query_2_items_parquet.to_parquet('query_2_items.parquet')

query_2_reviews_parquet = user_reviews_df[['user_id','recommend']]
query_2_reviews_parquet.to_parquet('query_2_reviews.parquet')

In [75]:
userid = '76561197970982479'
# userid = 'erickoiv'
# userid = 'jjas01'

def read_parquets ():
    query_2_steam = pd.read_parquet('query_2_steam.parquet')
    query_2_reviews = pd.read_parquet('query_2_reviews.parquet')
    return query_2_steam, query_2_reviews

# Cantidad de dinero gastado por el usuario

def load_parquet_in_batches(file_path,user_id,chunk_size=5000):
  """Loads a Parquet file in batches.

  Args:
    file_path: The path to the Parquet file.
    chunk_size: The size of each batch.

  Returns:
    A list of Pandas DataFrames.
  """
  parquet_file = pq.ParquetFile(file_path)
  dataframes = []
  for batch in parquet_file.iter_batches(batch_size=5000):
      dataframes = batch.to_pandas()
      result = get_items_names(dataframes,user_id)
      if result is not None :
        return result
  return result

 
def get_items_names(items_dataframe,user_id):

    items_dataframe = items_dataframe.set_index("user_id")
    found_data_frame = items_dataframe.loc[user_id]
    if not found_data_frame.empty:
        return found_data_frame
    else:
        return None


def get_items_id(row_df):
    items= []
    user_items = row_df.iloc[0]

    for item in user_items:
        items.append(item['item_id'])
    return items

def get_waste(items_list):
    waste = 0
    prices = []
    for item in items_list:
        intintem=int(item)
        price = query_2_steam.query("id == @intintem")
        # prices.append((price['price'].values))

        if len(price['price'].values) > 0:
            waste = waste+price['price'].values[0]
            prices.append((price['price'].values))
    return waste


def percent_reviews(user_id):
    try:
        recommends = query_2_reviews.query("user_id == @user_id")
        recommends_true = query_2_reviews.query("(user_id == @user_id) &(recommend == True)")
        items_amount = len(recommends)
        percent = (len(recommends_true) * 100) / items_amount
        return percent, items_amount
    except ZeroDivisionError:
        return 0,0
    
# Load small Parquets
query_2_steam, query_2_reviews = read_parquets()

# Load the Parquet file in batches
query_2_items = load_parquet_in_batches('query_2_items.parquet',userid)

items_list = get_items_id(query_2_items)  

total_waste = get_waste(items_list)
reviews_items_percent, total_reviews = percent_reviews(userid)

json_output = {userid: { "spent_money":total_waste, "recommendation %":reviews_items_percent, "reviews_amount":total_reviews}}
json_output


0


{'jjas01': {'spent_money': 0, 'recommendation %': 60.0, 'reviews_amount': 10}}